# Protocol 1
InAs VLS pinch-off curves with standard lock-in technique [POC Ajuan]

## Imports

In [1]:
# Copy this to all notebooks!
from qcodes.logger import start_all_logging
start_all_logging()

# Import qcodes and other necessary packages
import qcodes as qc
import numpy as np
import time
from time import sleep
import matplotlib
import matplotlib.pyplot as plt

# Import device drivers 
from qcodes.instrument_drivers.QuantumDesign.DynaCoolPPMS import DynaCool
from qcodes.instrument_drivers.Keysight.Infiniium import Infiniium

# Import qcodes packages 
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.database import initialise_database
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)
from qcodes.dataset.experiment_container import load_experiment_by_name
from qcodes.instrument.base import Instrument
from qcodes import Station
from qcodes.dataset.data_export import flatten_1D_data_for_plot
from qcodes.utils.dataset.doNd import do1d,do2d



go = 7.7480917310e-5

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\200603-9288-qcodes.log
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


2020-06-03 11:02:22,690 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ C:\Users\lyn-ppmsmsr-01usr\miniconda3\envs\qcodes\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'



Qcodes Logfile : C:\Users\lyn-ppmsmsr-01usr\.qcodes\logs\200603-9288-qcodes.log


## Station
(Need to load 3 Keithleys and 6 Lock-In Amps)

In [2]:
# Create station, instantiate instruments
Instrument.close_all()
path_to_station_file = 'C:/Users/lyn-ppmsmsr-01usr/Desktop/station.yaml'
#                       'file//station.yaml'
# Here we load the station file.
station = Station()
station.load_config_file(path_to_station_file)

# Connect to ppms
#Instrument.find_instrument('ppms_cryostat')
ppms = DynaCool.DynaCool(name = "ppms_cryostat", address="TCPIP0::10.10.117.37::5000::SOCKET")
station.add_component(ppms)

# SRS
lockin_1 = station.load_instrument('lockin_1')
lockin_2 = station.load_instrument('lockin_2')
lockin_3 = station.load_instrument('lockin_3')
lockin_4 = station.load_instrument('lockin_4')
lockin_5 = station.load_instrument('lockin_5')
lockin_6 = station.load_instrument('lockin_6')

# DMMs
dmm_a = station.load_instrument('Keithley_A')
dmm_b = station.load_instrument('Keithley_B')
dmm_c = station.load_instrument('Keithley_C')

dmm_a.smua.volt(0) # Set voltages to 0
dmm_a.smub.volt(0) # Set voltages to 0

dmm_b.smua.volt(0) # Set voltages to 0
dmm_b.smub.volt(0) # Set voltages to 0

dmm_c.smua.volt(0) # Set voltages to 0
dmm_c.smub.volt(0) # Set voltages to 0

Connected to:  QuantumDesign dynacool (serial:N/A, firmware:N/A) in 0.39s
Connected to: Stanford_Research_Systems SR830 (serial:s/n91493, firmware:ver1.07) in 0.08s
Connected to: Stanford_Research_Systems SR830 (serial:s/n87831, firmware:ver1.07) in 0.04s
Connected to: Stanford_Research_Systems SR830 (serial:s/n91797, firmware:ver1.07) in 0.04s
Connected to: Stanford_Research_Systems SR830 (serial:s/n57026, firmware:ver1.07) in 0.04s
Connected to: Stanford_Research_Systems SR830 (serial:s/n91992, firmware:ver1.07) in 0.04s
Connected to: Stanford_Research_Systems SR830 (serial:s/n91973, firmware:ver1.07) in 0.04s
Connected to: Keithley Instruments Inc. 2614B (serial:4376445, firmware:3.2.2) in 0.14s
Connected to: Keithley Instruments Inc. 2612B (serial:4428183, firmware:3.3.5) in 0.02s
Connected to: Keithley Instruments Inc. 2614B (serial:4376447, firmware:3.2.2) in 0.02s


In [3]:
for inst in station.components.values():
    inst.print_readable_snapshot()

ppms_cryostat:
	parameter           value
--------------------------------------------------------------------------------
IDN                  :	{'vendor': ' QuantumDesign', 'model': 'dynacool', 'ser...
chamber_state        :	sealed 
chamber_temperature  :	1.6858 (K)
field_approach       :	linear 
field_measured       :	2.2216e-05 (T)
field_ramp           :	None (T)
field_rate           :	0 (T/s)
field_target         :	2.2216e-05 (T)
field_tolerance      :	0.0005 (T)
magnet_state         :	holding 
temperature          :	1.6858 (K)
temperature_rate     :	0 (K/s)
temperature_setpoint :	1.7 (K)
temperature_settling :	fast settle 
temperature_state    :	chasing 
timeout              :	5 (s)
lockin_1:
	parameter       value
--------------------------------------------------------------------------------
IDN              :	{'vendor': 'Stanford_Research_Systems', 'model': 'SR830', ...
P                :	0.816 (deg)
R                :	0.035652 (V)
R_offset         :	(0.0, 0) 
X              

## DB File, Location

In [4]:
#config.save_to_home  ### Initialize database, make new measurement
%matplotlib notebook

from qcodes.dataset.database import get_DB_location
from qcodes import config
import os.path
import os

mainpath = 'C:/Users/MicrosoftQ/Desktop/Results' #remember to add << /User_name >> to save the db file in your own user folder
config.current_config.core.db_location = os.path.join(mainpath,'QDevXXX_X_YYYYMMDD.db')
config.current_config
newpath = os.path.join(mainpath,'QDevXXX_X_YYYYMMDD') 
if not os.path.exists(newpath):
    os.makedirs(newpath)
figurepath = newpath
initialise_database()

## Functions

In [ ]:
def wait_for_field():
    time.sleep(1)
    Magnet_state = ppms.magnet_state()
    while Magnet_state is not 'holding':
        #print('waiting for field')
        time.sleep(0.1)
        Magnet_state = ppms.magnet_state()
        #print('field ready')
    return

def wait_for_field_ramp():
    Magnet_state = ppms.magnet_state()
    while Magnet_state is not 'ramping':
        time.sleep(1)
        Magnet_state = ppms.magnet_state()
    return

def field_ready():
    return ppms.magnet_state() == 'holding'

def wait_for_temp():
    Temp_state = ppms.temperature_state()
    while Temp_state is not 'stable':
        time.sleep(1)
        Temp_state = ppms.temperature_state()
    return

def wait_for_near_temp():
    Temp_state = ppms.temperature_state()
    while Temp_state is not 'near':
        time.sleep(2)
        Temp_state = ppms.temperature_state()
    time.sleep(10)
    return

## Lock-in add-on functions
Gains and conductance

In [ ]:
# Add the gain property function on each individual lock-in

GI1 = 1e6
GV2 = 5e2

def gains(GI = GI, GV = GV):
    return GI, GV


try:
    lockin_1.add_parameter("gain_I", label="GIamp", unit="A/V", get_cmd = gains[0])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_1.parameters['gain_I']
try:
    lockin_2.add_parameter("gain_V", label="GIamp", unit="", get_cmd = gains[1])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_2.parameters['gain_V']
try:
    lockin_3.add_parameter("gain_I", label="GIamp", unit="A/V", get_cmd = gains[0])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_3.parameters['gain_I']
try:
    lockin_4.add_parameter("gain_V", label="GIamp", unit="", get_cmd = gains[1])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_4.parameters['gain_V']
try:
    lockin_5.add_parameter("gain_I", label="GIamp", unit="A/V", get_cmd = gains[0])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_5.parameters['gain_I']
try:
    lockin_6.add_parameter("gain_V", label="GIamp", unit="", get_cmd = gains[1])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_6.parameters['gain_V']

In [405]:
# DIFFERENTIAL CONDUCTANCE FOR A FOUR PROBE MEASUREMENT_with_voltage amplifier 
# Lock-ins 1(current), 2(voltage)

def desoverh_fpm():    
    I_fpm = lockin_1.R()/lockin_1.gain_I()
    V_fpm = lockin_2.R()/lockin_2.gain_V()
    
    
    if V_fpm == 0:
        dcond_fpm = 0
    else:
        dcond_fpm = I_fpm/V_fpm/go
    
    
    if I_fpm== 0:
        res_fpm = 0
    else:
        res_fpm = V_fpm/I_fpm
    
    
    return dcond_fpm, res_fpm
    
try:
    lockin_1.add_parameter("diff_conductance_fpm", label="dI/dV", unit="2e^2/h", get_cmd = desoverh_fpm[0])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_1.parameters['diff_conductance_fpm']
    
try:
    lockin_1.add_parameter("resistance_fpm", label="R", unit="Ohm", get_cmd = desoverh_fpm[1])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_1.parameters['resistance_fpm']

In [ ]:
# Lock-ins 3(current), 4(voltage)

def desoverh_fpm():    
    I_fpm = lockin_3.R()/lockin_3.gain_I()
    V_fpm = lockin_4.R()/lockin_4.gain_V()
    
    
    if V_fpm == 0:
        dcond_fpm = 0
    else:
        dcond_fpm = I_fpm/V_fpm/go
    
    
    if I_fpm== 0:
        res_fpm = 0
    else:
        res_fpm = V_fpm/I_fpm
    
    
    return dcond_fpm, res_fpm
    
try:
    lockin_3.add_parameter("diff_conductance_fpm", label="dI/dV", unit="2e^2/h", get_cmd = desoverh_fpm[0])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_3.parameters['diff_conductance_fpm']
    
try:
    lockin_4.add_parameter("resistance_fpm", label="R", unit="Ohm", get_cmd = desoverh_fpm[1])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_4.parameters['resistance_fpm']

In [ ]:
# Lock-ins 5(current), 6(voltage)

def desoverh_fpm():    
    I_fpm = lockin_5.R()/lockin_5.gain_I()
    V_fpm = lockin_6.R()/lockin_6.gain_V()
    
    
    if V_fpm == 0:
        dcond_fpm = 0
    else:
        dcond_fpm = I_fpm/V_fpm/go
    
    
    if I_fpm== 0:
        res_fpm = 0
    else:
        res_fpm = V_fpm/I_fpm
    
    
    return dcond_fpm, res_fpm
    
try:
    lockin_5.add_parameter("diff_conductance_fpm", label="dI/dV", unit="2e^2/h", get_cmd = desoverh_fpm[0])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_5.parameters['diff_conductance_fpm']
    
try:
    lockin_6.add_parameter("resistance_fpm", label="R", unit="Ohm", get_cmd = desoverh_fpm[1])
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_6.parameters['resistance_fpm']

## Measurement parameters

In [ ]:
ACdiv = 1e-4
DCdiv = 1e-2

GIamp = 1e6 #A/V    [need to be tested and specified]
GVamp = 500  #     [need to be tested and specified]

Vgmin = -30 #V    [need to be tested and specified]
Vgmax = +30 #V    [need to be tested and specified]
Npoints = 301 #  [need to be tested and specified] 


VSD   = 0 #V        [need to be tested and specified]
timedelay = 1 # sec   [need to be tested and specified]


VAC   = 1 #V     [need to be tested and specified]
f     = 18 # or 120 or 67 Hz [need to be tested and specified]
tcI   = 0.3  #sec     [need to be tested and specified]
tcV   = 0.3  #sec     [need to be tested and specified]
dB_slope  = 12  # dB     [need to be tested and specified]


temperature = 1.7 #K
temperature_rate = 0.1
magnetic_field = 0 #T
magnetic_field_rate = 1

N = 3 #           Number of repetitions

In [378]:
# Set things up to the station

lockin_1.time_constant(tcI)  # set time constant on the lock-in
lockin_1.sensitivity(50e-3)      # set sensitivity on the lock-in
lockin_1.frequency(f)          # set frequency on the lock-in
lockin_1.amplitude(VAC)          # set amplitude on the lock-in
lockin_1.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_2.time_constant(tcV)  # set time constant on the lock-in
lockin_2.sensitivity(20e-3)      # set sensitivity on the lock-in
lockin_2.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_3.time_constant(tcI)  # set time constant on the lock-in
lockin_3.sensitivity(50e-3)      # set sensitivity on the lock-in
lockin_3.frequency(f)          # set frequency on the lock-in
lockin_3.amplitude(VAC)          # set amplitude on the lock-in
lockin_3.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_4.time_constant(tcV)  # set time constant on the lock-in
lockin_4.sensitivity(20e-3)      # set sensitivity on the lock-in
lockin_4.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_5.time_constant(tcI)  # set time constant on the lock-in
lockin_5.sensitivity(50e-3)      # set sensitivity on the lock-in
lockin_5.frequency(f)          # set frequency on the lock-in
lockin_5.amplitude(VAC)          # set amplitude on the lock-in
lockin_5.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_6.time_constant(tcV)  # set time constant on the lock-in
lockin_6.sensitivity(20e-3)      # set sensitivity on the lock-in
lockin_6.filter_slope(dB_slope)    # set filter slope on the lock-in


dcond1 = lockin_1.diff_conductance_fpm
res1   = lockin_1.resistance_fpm
ampl1  = lockin_1.R

dcond3 = lockin_3.diff_conductance_fpm
res3   = lockin_3.resistance_fpm
ampl3  = lockin_3.R

dcond5 = lockin_5.diff_conductance_fpm
res5   = lockin_5.resistance_fpm
ampl5  = lockin_5.R

gate   = dmm_a.smub.volt
bias1  = dmm_a.smua.volt
bias3  = dmm_c.smua.volt
bias5  = dmm_c.smub.volt

temp      = ppms.temperature            # to read the temperature
temp_set  = ppms.temperature_setpoint   # to set the temperature
temp_rate = ppms.temperature_rate       # to set the temperature rate


temp_rate(temperature_rate)
temp_set(temperature)


field       = ppms.field_measured        # to read the magnetic field
field_set   = ppms.field_ramp            # to set the magnetic field
field_rate  = ppms.field_rate            # to set the the magnetic field rate


field_rate(magnetic_field_rate)
field_set(magnetic_field)

## The measurement
A control parameter is included for sequence measuerements. Control parameter could be bias, field, temperature

In [435]:
dmm_a.smub.output('on') # the gate
dmm_a.smua.output('on') # the bias for 1
dmm_b.smua.output('on') # the bias for 3
dmm_b.smub.output('on') # the bias for 5

# in this case control parameter is teperature
paramrange = [2.7,5] #arbitrary values

# Run
for var_param in paramrange:
    
    #bias1(paramrange/DCdiv)
    #bias3(paramrange/DCdiv)    
    #bias5(paramrange/DCdiv)
    
    #magnetic_field = var_param
    #ppms.field_ramp(magnetic_field)
    #wait_for_field()
    
    temperature = var_param    
    ppms.temperature_setpoint(temperature)
    wait_for_temp() 

    Experiment_name = "Protocol_1_ramp_to-Vgmin_"+"T="+"{:.2f}".format(temperature)+"K_B"+"{:.2f}".format(magnetic_field)+"T"
    Sample_name = "QDevXXX_X_1:DeviceA_3:DeviceB_5:DeviceC"
    
    new_experiment(name=Experiment_name, sample_name = Sample_name)
    do1d(gate,0,Vgmin,Npoints,timedelay,
         dcond1,ampl1,
         dcond3,ampl3,
         dcond5,ampl5,
         do_plot=False)
    
    
    for i in N:
        Experiment_name = "Protocol_1_"+"itteretion"+"{:.0f}".format(N)+"T="+"{:.2f}".format(temperature)+"K_B"+"{:.2f}".format(magnetic_field)+"T"
        Sample_name = "QDevXXX_X_1:DeviceA_3:DeviceB_5:DeviceC"

        new_experiment(name=Experiment_name+'UP', sample_name = Sample_name)
        do1d(gate,Vgmin,Vgmax,Npoints,timedelay,
             dcond1,ampl1,
             dcond3,ampl3,
             dcond5,ampl5,
             do_plot=False)

        new_experiment(name=Experiment_name+'DOWN', sample_name = Sample_name)
        do1d(gate,Vgmax,Vgmin,Npoints,timedelay,
             dcond1,ampl1,
             dcond3,ampl3,
             dcond5,ampl5,
             do_plot=False)

    Experiment_name = "Protocol_1_ramp_toZero_"+"T="+"{:.2f}".format(temperature)+"K_B"+"{:.2f}".format(magnetic_field)+"T"
    Sample_name = "QDevXXX_X_1:DeviceA_3:DeviceB_5:DeviceC"
    
    new_experiment(name=Experiment_name, sample_name = Sample_name)
    do1d(gate,Vgmin,0,Npoints,timedelay,
         dcond1,ampl1,
         dcond3,ampl3,
         dcond5,ampl5,
         do_plot=False)
    
    
    
    
    
    
dmm_a.smub.output('off') # the gate
dmm_a.smua.output('off') # the bias for 1
dmm_b.smua.output('off') # the bias for 3
dmm_b.smub.output('off') # the bias for 5

Turning Keithley channels on
Setting gate to zero and ready to ramp
Vg=0.0 V,-1.0 V,-2.0 V,-3.0 V,-4.0 V,-5.0 V,-6.0 V,-7.0 V,-8.0 V,-9.0 V,-10.0 V,-11.0 V,-12.0 V,-13.0 V,-14.0 V,-15.0 V,-16.0 V,-17.0 V,-18.0 V,-19.0 V,-20.0 V,-21.0 V,-22.0 V,-23.0 V,-24.0 V,-25.0 V,-26.0 V,-27.0 V,-28.0 V,-29.0 V,-30.0 V,Starting experimental run with id: 305. 
Starting experimental run with id: 306. 
Experiment is done; Gate is ramping back to zero
Vg=30.0 V,29.0 V,28.0 V,27.0 V,26.0 V,25.0 V,24.0 V,23.0 V,22.0 V,21.0 V,20.0 V,19.0 V,18.0 V,17.0 V,16.0 V,15.0 V,14.0 V,13.0 V,12.0 V,11.0 V,10.0 V,9.0 V,8.0 V,7.0 V,6.0 V,5.0 V,4.0 V,3.0 V,2.0 V,1.0 V,0.0 V,Turning Keithley channels off
